In [1]:
import pandas as pd
from scipy.io import arff

import os
import sys

# Zakładamy, że foldery 'exception-rules' i 'decision-rules' są w katalogu nadrzędnym względem aktualnego working directory
notebook_dir = os.getcwd()

exception_rules_path = os.path.abspath(os.path.join(notebook_dir, '..',  '..', 'exception-rules'))
decision_rules_path = os.path.abspath(os.path.join(notebook_dir, '..',  '..', 'decision-rules'))

sys.path.append(exception_rules_path)
sys.path.append(decision_rules_path)

In [2]:
from exception_rules.measures import *

## Our aproach

In [3]:
df = pd.DataFrame(arff.loadarff(f"./mushroom.arff")[0])
# code to change encoding of the file
tmp_df = df.select_dtypes([object])
tmp_df = tmp_df.stack().str.decode("utf-8").unstack()
for col in tmp_df:
    df[col] = tmp_df[col].replace({'?': None})
    
X = df.drop(columns=["class"])
y = df["class"]

In [4]:
with open(f"./txt/Our_approach_rules.txt") as file:
    ruleset_text = file.readlines()

In [5]:
from decision_rules.ruleset_factories._factories.classification.text_factory import TextRuleSetFactory

In [6]:
factory = TextRuleSetFactory()

ruleset = factory.make(ruleset_text, X, y)

In [7]:
for rule in ruleset.rules:
    print(rule)

IF odor = {f} THEN class = p (p=2160, n=0, P=3916, N=4208)
IF gill-color = {b} THEN class = p (p=1728, n=0, P=3916, N=4208)
IF stalk-surface-above-ring = {k} AND gill-spacing = {c} THEN class = p (p=2228, n=0, P=3916, N=4208)
IF gill-size = {n} THEN class = p (p=2224, n=288, P=3916, N=4208)
IF spore-print-color = {h} THEN class = p (p=1584, n=48, P=3916, N=4208)
IF gill-size = {n} AND spore-print-color = {h} THEN class = e (p=48, n=0, P=4208, N=3916)
IF spore-print-color = {r} THEN class = p (p=72, n=0, P=3916, N=4208)
IF odor = {n} THEN class = e (p=3408, n=120, P=4208, N=3916)
IF stalk-root = {c} THEN class = e (p=512, n=44, P=4208, N=3916)
IF odor = {n} AND stalk-root = {c} THEN class = p (p=8, n=0, P=3916, N=4208)
IF odor = {a} THEN class = e (p=400, n=0, P=4208, N=3916)
IF odor = {l} THEN class = e (p=400, n=0, P=4208, N=3916)


In [8]:
CR_rules = []
CR_rules.append(ruleset.rules[0])
CR_rules.append(ruleset.rules[1])
CR_rules.append(ruleset.rules[2])
CR_rules.append(ruleset.rules[3])
CR_rules.append(ruleset.rules[6])
CR_rules.append(ruleset.rules[7])

CR_rules.append(ruleset.rules[10])
CR_rules.append(ruleset.rules[11])

CR_rules[3].reference_rule = ruleset.rules[4]
CR_rules[3].exception_rule = ruleset.rules[5]
CR_rules[5].reference_rule = ruleset.rules[8]
CR_rules[5].exception_rule = ruleset.rules[9]

In [9]:
for rule in CR_rules:
    if rule.exception_rule is not None:
        GACE = calculate_GACE(rule, rule.exception_rule , return_ACE = False)
        RI = calculate_RI(rule, rule.exception_rule , rule.reference_rule)
        RES = calculate_my_measure(rule, rule.exception_rule)

        print(f"CR: {rule}, GACE: {round(GACE,3)}, RI: {round(RI,3)}, RES: {round(RES,3)}")
        print(f"RR: {rule.reference_rule} ")
        print(f"ER: {rule.exception_rule} ")
        print("")
    else:
        print(f"CR: {rule}")
        print("")


CR: IF odor = {f} THEN class = p (p=2160, n=0, P=3916, N=4208)

CR: IF gill-color = {b} THEN class = p (p=1728, n=0, P=3916, N=4208)

CR: IF stalk-surface-above-ring = {k} AND gill-spacing = {c} THEN class = p (p=2228, n=0, P=3916, N=4208)

CR: IF gill-size = {n} THEN class = p (p=2224, n=288, P=3916, N=4208), GACE: 0.03, RI: 3.164, RES: 0.576
RR: IF spore-print-color = {h} THEN class = p (p=1584, n=48, P=3916, N=4208) 
ER: IF gill-size = {n} AND spore-print-color = {h} THEN class = e (p=48, n=0, P=4208, N=3916) 

CR: IF spore-print-color = {r} THEN class = p (p=72, n=0, P=3916, N=4208)

CR: IF odor = {n} THEN class = e (p=3408, n=120, P=4208, N=3916), GACE: 0.018, RI: 4.996, RES: 0.672
RR: IF stalk-root = {c} THEN class = e (p=512, n=44, P=4208, N=3916) 
ER: IF odor = {n} AND stalk-root = {c} THEN class = p (p=8, n=0, P=3916, N=4208) 

CR: IF odor = {a} THEN class = e (p=400, n=0, P=4208, N=3916)

CR: IF odor = {l} THEN class = e (p=400, n=0, P=4208, N=3916)



c:\Users\dawid\Desktop\exception_rules_repo\exception-rules\exception-rules\exception_rules\measures.py:11: RuntimeWarning: divide by zero encountered in log2
  not_X = cov.n/(cov.P + cov.N) * np.log2((cov.n/(cov.p+cov.n))/(cov.N/(cov.P+cov.N)))
c:\Users\dawid\Desktop\exception_rules_repo\exception-rules\exception-rules\exception_rules\measures.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  not_X = cov.n/(cov.P + cov.N) * np.log2((cov.n/(cov.p+cov.n))/(cov.N/(cov.P+cov.N)))


In [10]:
with open("txt_results/Our_approach_rules.txt", "w", encoding="utf-8") as file:
    for rule in CR_rules:
        if rule.exception_rule is not None:
            GACE = calculate_GACE(rule, rule.exception_rule, return_ACE=False)
            RI = calculate_RI(rule, rule.exception_rule, rule.reference_rule)
            RES = calculate_my_measure(rule, rule.exception_rule)

            file.write(f"CR: {rule}, GACE: {round(GACE,3)}, RI: {round(RI,3)}, RES: {round(RES,3)}\n")
            file.write(f"RR: {rule.reference_rule} \n")
            file.write(f"ER: {rule.exception_rule} \n")
            file.write("\n")
        else:
            file.write(f"CR: {rule}\n")
            file.write("\n")

## MEPRO

In [11]:
df = pd.DataFrame(arff.loadarff(f"./mushroom.arff")[0])
# code to change encoding of the file
tmp_df = df.select_dtypes([object])
tmp_df = tmp_df.stack().str.decode("utf-8").unstack()
for col in tmp_df:
    df[col] = tmp_df[col]
    
X = df.drop(columns=["class"])
y = df["class"]

In [12]:
with open(f"./txt/MEPRO_rules.txt") as file:
    ruleset_text = file.readlines()

In [13]:
from decision_rules.ruleset_factories._factories.classification.text_factory import TextRuleSetFactory

In [14]:
factory = TextRuleSetFactory()

ruleset = factory.make(ruleset_text, X, y)

In [15]:
for rule in ruleset.rules:
    print(rule)

IF bruises = {f} AND gill-attachment = {f} AND ring-number = {o} THEN class = p (p=3256, n=960, P=3916, N=4208)
IF odor = {n} AND stalk-color-below-ring = {w} THEN class = e (p=1904, n=80, P=4208, N=3916)
IF bruises = {f} AND gill-attachment = {f} AND ring-number = {o} AND odor = {n} AND stalk-color-below-ring = {w} THEN class = e (p=912, n=0, P=4208, N=3916)
IF bruises = {f} AND veil-color = {w} AND ring-number = {o} THEN class = p (p=3248, n=960, P=3916, N=4208)
IF odor = {n} AND stalk-color-below-ring = {w} THEN class = e (p=1904, n=80, P=4208, N=3916)
IF bruises = {f} AND veil-color = {w} AND ring-number = {o} AND odor = {n} AND stalk-color-below-ring = {w} THEN class = e (p=912, n=0, P=4208, N=3916)
IF gill-size = {b} THEN class = e (p=3920, n=1692, P=4208, N=3916)
IF odor = {f} THEN class = p (p=2160, n=0, P=3916, N=4208)
IF gill-size = {b} AND odor = {f} THEN class = p (p=1584, n=0, P=3916, N=4208)
IF gill-size = {b} THEN class = e (p=3920, n=1692, P=4208, N=3916)
IF spore-print

In [16]:
CR_rules = []

for i in range(0, len(ruleset.rules), 3):
    ruleset.rules[i].reference_rule = ruleset.rules[i + 1]
    ruleset.rules[i].exception_rule = ruleset.rules[i + 2]
    CR_rules.append(ruleset.rules[i])


In [17]:
for rule in CR_rules:
    if rule.exception_rule is not None:
        GACE = calculate_GACE(rule, rule.exception_rule , return_ACE = False)
        RI = calculate_RI(rule, rule.exception_rule , rule.reference_rule)
        RES = calculate_my_measure(rule, rule.exception_rule)

        print(f"CR: {rule}, GACE: {round(GACE,3)}, RI: {round(RI,3)}, RES: {round(RES,3)}")
        print(f"RR: {rule.reference_rule} ")
        print(f"ER: {rule.exception_rule} ")
        print("")

    else:
        print(f"CR: {rule}")
        print("")

CR: IF bruises = {f} AND gill-attachment = {f} AND ring-number = {o} THEN class = p (p=3256, n=960, P=3916, N=4208), GACE: 0.119, RI: 2.367, RES: 0.903
RR: IF odor = {n} AND stalk-color-below-ring = {w} THEN class = e (p=1904, n=80, P=4208, N=3916) 
ER: IF bruises = {f} AND gill-attachment = {f} AND ring-number = {o} AND odor = {n} AND stalk-color-below-ring = {w} THEN class = e (p=912, n=0, P=4208, N=3916) 

CR: IF bruises = {f} AND veil-color = {w} AND ring-number = {o} THEN class = p (p=3248, n=960, P=3916, N=4208), GACE: 0.118, RI: 2.364, RES: 0.902
RR: IF odor = {n} AND stalk-color-below-ring = {w} THEN class = e (p=1904, n=80, P=4208, N=3916) 
ER: IF bruises = {f} AND veil-color = {w} AND ring-number = {o} AND odor = {n} AND stalk-color-below-ring = {w} THEN class = e (p=912, n=0, P=4208, N=3916) 

CR: IF gill-size = {b} THEN class = e (p=3920, n=1692, P=4208, N=3916), GACE: 0.117, RI: 2.015, RES: 0.95
RR: IF odor = {f} THEN class = p (p=2160, n=0, P=3916, N=4208) 
ER: IF gill-si

c:\Users\dawid\Desktop\exception_rules_repo\exception-rules\exception-rules\exception_rules\measures.py:11: RuntimeWarning: divide by zero encountered in log2
  not_X = cov.n/(cov.P + cov.N) * np.log2((cov.n/(cov.p+cov.n))/(cov.N/(cov.P+cov.N)))
c:\Users\dawid\Desktop\exception_rules_repo\exception-rules\exception-rules\exception_rules\measures.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  not_X = cov.n/(cov.P + cov.N) * np.log2((cov.n/(cov.p+cov.n))/(cov.N/(cov.P+cov.N)))


In [18]:
with open("txt_results/MEPRO_rules.txt", "w", encoding="utf-8") as file:
    for rule in CR_rules:
        if rule.exception_rule is not None:
            GACE = calculate_GACE(rule, rule.exception_rule, return_ACE=False)
            RI = calculate_RI(rule, rule.exception_rule, rule.reference_rule)
            RES = calculate_my_measure(rule, rule.exception_rule)

            file.write(f"CR: {rule}, GACE: {round(GACE,3)}, RI: {round(RI,3)}, RES: {round(RES,3)}\n")
            file.write(f"RR: {rule.reference_rule} \n")
            file.write(f"ER: {rule.exception_rule} \n")
            file.write("\n")
        else:
            file.write(f"CR: {rule}\n")
            file.write("\n")

## MEPROUX

In [19]:
df = pd.DataFrame(arff.loadarff(f"./mushroom.arff")[0])
# code to change encoding of the file
tmp_df = df.select_dtypes([object])
tmp_df = tmp_df.stack().str.decode("utf-8").unstack()
for col in tmp_df:
    df[col] = tmp_df[col]
    
X = df.drop(columns=["class"])
y = df["class"]

In [20]:
with open(f"./txt/MEPROUX_rules.txt") as file:
    ruleset_text = file.readlines()

In [21]:
from decision_rules.ruleset_factories._factories.classification.text_factory import TextRuleSetFactory

In [22]:
factory = TextRuleSetFactory()

ruleset = factory.make(ruleset_text, X, y)

In [23]:
for rule in ruleset.rules:
    print(rule)

IF bruises = {f} AND gill-attachment = {f} AND ring-number = {o} THEN class = p (p=3256, n=960, P=3916, N=4208)
IF stalk-surface-above-ring = {f} THEN class = e (p=408, n=144, P=4208, N=3916)
IF bruises = {f} AND gill-attachment = {f} AND ring-number = {o} AND stalk-surface-above-ring = {f} THEN class = e (p=408, n=0, P=4208, N=3916)
IF bruises = {f} AND veil-color = {w} AND ring-number = {o} THEN class = p (p=3248, n=960, P=3916, N=4208)
IF stalk-surface-above-ring = {f} THEN class = e (p=408, n=144, P=4208, N=3916)
IF bruises = {f} AND veil-color = {w} AND ring-number = {o} AND stalk-surface-above-ring = {f} THEN class = e (p=408, n=0, P=4208, N=3916)
IF bruises = {f} AND ring-number = {o} THEN class = e (p=1152, n=3256, P=4208, N=3916)
IF stalk-surface-above-ring = {f} THEN class = e (p=408, n=144, P=4208, N=3916)
IF bruises = {f} AND ring-number = {o} AND stalk-surface-above-ring = {f} THEN class = e (p=408, n=0, P=4208, N=3916)
IF bruises = {f} AND veil-color = {w} THEN class = p 

In [24]:
CR_rules = []

for i in range(0, len(ruleset.rules), 3):
    ruleset.rules[i].reference_rule = ruleset.rules[i + 1]
    ruleset.rules[i].exception_rule = ruleset.rules[i + 2]
    CR_rules.append(ruleset.rules[i])

In [25]:
for rule in CR_rules:
    if rule.exception_rule is not None:
        GACE = calculate_GACE(rule, rule.exception_rule , return_ACE = False)
        RI = calculate_RI(rule, rule.exception_rule , rule.reference_rule)
        RES = calculate_my_measure(rule, rule.exception_rule)

        print(f"CR: {rule}, GACE: {round(GACE,3)}, RI: {round(RI,3)}, RES: {round(RES,3)}")
        print(f"RR: {rule.reference_rule} ")
        print(f"ER: {rule.exception_rule} ")
        print("")

    else:
        print(f"CR: {rule}")
        print("")

CR: IF bruises = {f} AND gill-attachment = {f} AND ring-number = {o} THEN class = p (p=3256, n=960, P=3916, N=4208), GACE: 0.079, RI: 2.68, RES: 0.772
RR: IF stalk-surface-above-ring = {f} THEN class = e (p=408, n=144, P=4208, N=3916) 
ER: IF bruises = {f} AND gill-attachment = {f} AND ring-number = {o} AND stalk-surface-above-ring = {f} THEN class = e (p=408, n=0, P=4208, N=3916) 

CR: IF bruises = {f} AND veil-color = {w} AND ring-number = {o} THEN class = p (p=3248, n=960, P=3916, N=4208), GACE: 0.079, RI: 2.677, RES: 0.771
RR: IF stalk-surface-above-ring = {f} THEN class = e (p=408, n=144, P=4208, N=3916) 
ER: IF bruises = {f} AND veil-color = {w} AND ring-number = {o} AND stalk-surface-above-ring = {f} THEN class = e (p=408, n=0, P=4208, N=3916) 

CR: IF bruises = {f} AND ring-number = {o} THEN class = e (p=1152, n=3256, P=4208, N=3916), GACE: 0.071, RI: 0.894, RES: 0.418
RR: IF stalk-surface-above-ring = {f} THEN class = e (p=408, n=144, P=4208, N=3916) 
ER: IF bruises = {f} AND 

c:\Users\dawid\Desktop\exception_rules_repo\exception-rules\exception-rules\exception_rules\measures.py:11: RuntimeWarning: divide by zero encountered in log2
  not_X = cov.n/(cov.P + cov.N) * np.log2((cov.n/(cov.p+cov.n))/(cov.N/(cov.P+cov.N)))
c:\Users\dawid\Desktop\exception_rules_repo\exception-rules\exception-rules\exception_rules\measures.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  not_X = cov.n/(cov.P + cov.N) * np.log2((cov.n/(cov.p+cov.n))/(cov.N/(cov.P+cov.N)))


In [26]:
with open("txt_results/MEPROUX_rules.txt", "w", encoding="utf-8") as file:
    for rule in CR_rules:
        if rule.exception_rule is not None:
            GACE = calculate_GACE(rule, rule.exception_rule, return_ACE=False)
            RI = calculate_RI(rule, rule.exception_rule, rule.reference_rule)
            RES = calculate_my_measure(rule, rule.exception_rule)

            file.write(f"CR: {rule}, GACE: {round(GACE,3)}, RI: {round(RI,3)}, RES: {round(RES,3)}\n")
            file.write(f"RR: {rule.reference_rule} \n")
            file.write(f"ER: {rule.exception_rule} \n")
            file.write("\n")
        else:
            file.write(f"CR: {rule}\n")
            file.write("\n")

## Hussain

In [27]:
df = pd.DataFrame(arff.loadarff(f"./mushroom.arff")[0])
# code to change encoding of the file
tmp_df = df.select_dtypes([object])
tmp_df = tmp_df.stack().str.decode("utf-8").unstack()
for col in tmp_df:
    df[col] = tmp_df[col]
    
X = df.drop(columns=["class"])
y = df["class"]

In [28]:
with open(f"./txt/Hussain_rules.txt") as file:
    ruleset_text = file.readlines()

In [29]:
from decision_rules.ruleset_factories._factories.classification.text_factory import TextRuleSetFactory

In [30]:
factory = TextRuleSetFactory()

ruleset = factory.make(ruleset_text, X, y)

In [31]:
for rule in ruleset.rules:
    print(rule)

IF stalk-root = {?} THEN class = p (p=1760, n=720, P=3916, N=4208)
IF bruises = {f} AND gill-size = {b} AND stalk-shape = {e} AND veil-type = {p} THEN class = e (p=496, n=1332, P=4208, N=3916)
IF stalk-root = {?} AND bruises = {f} AND gill-size = {b} AND stalk-shape = {e} AND veil-type = {p} THEN class = e (p=480, n=0, P=4208, N=3916)
IF ring-type = {e} THEN class = p (p=1768, n=1008, P=3916, N=4208)
IF bruises = {f} AND gill-size = {b} AND ring-number = {t} THEN class = e (p=304, n=0, P=4208, N=3916)
IF ring-type = {e} AND bruises = {f} AND gill-size = {b} AND ring-number = {t} THEN class = e (p=0, n=0, P=4208, N=3916)
IF gill-color = {p} AND veil-type = {p} AND veil-color = {w} THEN class = p (p=640, n=852, P=3916, N=4208)
IF stalk-root = {?} THEN class = e (p=720, n=1760, P=4208, N=3916)
IF gill-color = {p} AND veil-type = {p} AND veil-color = {w} AND stalk-root = {?} THEN class = e (p=108, n=0, P=4208, N=3916)
IF gill-attachment = {f} AND stalk-shape = {e} AND veil-type = {p} THEN 

In [32]:
CR_rules = []

for i in range(0, len(ruleset.rules), 3):
    ruleset.rules[i].exception_rule = ruleset.rules[i + 2]
    ruleset.rules[i].reference_rule = ruleset.rules[i + 1]
    CR_rules.append(ruleset.rules[i])

In [33]:
for rule in CR_rules:
    if rule.exception_rule is not None:
        GACE = calculate_GACE(rule, rule.exception_rule , return_ACE = False)
        RI = calculate_RI(rule, rule.exception_rule , rule.reference_rule)
        RES = calculate_my_measure(rule, rule.exception_rule)

        print(f"CR: {rule}, GACE: {round(GACE,3)}, RI: {round(RI,3)}, RES: {round(RES,3)}")
        print(f"RR: {rule.reference_rule} ")
        print(f"ER: {rule.exception_rule} ")
        print("")

    else:
        print(f"CR: {rule}")
        print("")

CR: IF stalk-root = {?} THEN class = p (p=1760, n=720, P=3916, N=4208), GACE: 0.051, RI: 3.818, RES: 0.641
RR: IF bruises = {f} AND gill-size = {b} AND stalk-shape = {e} AND veil-type = {p} THEN class = e (p=496, n=1332, P=4208, N=3916) 
ER: IF stalk-root = {?} AND bruises = {f} AND gill-size = {b} AND stalk-shape = {e} AND veil-type = {p} THEN class = e (p=480, n=0, P=4208, N=3916) 

CR: IF ring-type = {e} THEN class = p (p=1768, n=1008, P=3916, N=4208), GACE: 0.0, RI: 0.0, RES: 0.226
RR: IF bruises = {f} AND gill-size = {b} AND ring-number = {t} THEN class = e (p=304, n=0, P=4208, N=3916) 
ER: IF ring-type = {e} AND bruises = {f} AND gill-size = {b} AND ring-number = {t} THEN class = e (p=0, n=0, P=4208, N=3916) 

CR: IF gill-color = {p} AND veil-type = {p} AND veil-color = {w} THEN class = p (p=640, n=852, P=3916, N=4208), GACE: 0.004, RI: 2.589, RES: 0.363
RR: IF stalk-root = {?} THEN class = e (p=720, n=1760, P=4208, N=3916) 
ER: IF gill-color = {p} AND veil-type = {p} AND veil-co

c:\Users\dawid\Desktop\exception_rules_repo\exception-rules\exception-rules\exception_rules\measures.py:11: RuntimeWarning: divide by zero encountered in log2
  not_X = cov.n/(cov.P + cov.N) * np.log2((cov.n/(cov.p+cov.n))/(cov.N/(cov.P+cov.N)))
c:\Users\dawid\Desktop\exception_rules_repo\exception-rules\exception-rules\exception_rules\measures.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  not_X = cov.n/(cov.P + cov.N) * np.log2((cov.n/(cov.p+cov.n))/(cov.N/(cov.P+cov.N)))


In [34]:
with open("txt_results/Hussain_rules.txt", "w", encoding="utf-8") as file:
    for rule in CR_rules:
        if rule.exception_rule is not None:
            GACE = calculate_GACE(rule, rule.exception_rule, return_ACE=False)
            RI = calculate_RI(rule, rule.exception_rule, rule.reference_rule)
            RES = calculate_my_measure(rule, rule.exception_rule)

            file.write(f"CR: {rule}, GACE: {round(GACE,3)}, RI: {round(RI,3)}, RES: {round(RES,3)}\n")
            file.write(f"RR: {rule.reference_rule} \n")
            file.write(f"ER: {rule.exception_rule} \n")
            file.write("\n")
        else:
            file.write(f"CR: {rule}\n")
            file.write("\n")

## Ant colony

In [35]:
df = pd.DataFrame(arff.loadarff(f"./mushroom.arff")[0])
# code to change encoding of the file
tmp_df = df.select_dtypes([object])
tmp_df = tmp_df.stack().str.decode("utf-8").unstack()
for col in tmp_df:
    df[col] = tmp_df[col].replace({'?': None})
    
X = df.drop(columns=["class"])
y = df["class"]

In [36]:
with open(f"./txt/Ant_colony_rules.txt") as file:
    ruleset_text = file.readlines()

In [37]:
from decision_rules.ruleset_factories._factories.classification.text_factory import TextRuleSetFactory

In [38]:
factory = TextRuleSetFactory()

ruleset = factory.make(ruleset_text, X, y)

In [39]:
for rule in ruleset.rules:
    print(rule)

IF odor = {n} THEN class = e (p=3408, n=120, P=4208, N=3916)
IF stalk-color-above-ring = {y} THEN class = p (p=8, n=0, P=3916, N=4208)
IF odor = {n} AND stalk-color-above-ring = {y} THEN class = p (p=8, n=0, P=3916, N=4208)
IF stalk-root = {c} THEN class = e (p=512, n=44, P=4208, N=3916)
IF ring-type = {n} THEN class = p (p=36, n=0, P=3916, N=4208)
IF stalk-root = {c} AND ring-type = {n} THEN class = p (p=36, n=0, P=3916, N=4208)
IF bruises = {t} THEN class = e (p=2752, n=624, P=4208, N=3916)
IF spore-print-color = {r} THEN class = p (p=72, n=0, P=3916, N=4208)
IF bruises = {t} AND spore-print-color = {r} THEN class = p (p=72, n=0, P=3916, N=4208)
IF odor = {f} THEN class = p (p=2160, n=0, P=3916, N=4208)
IF odor = {p} THEN class = p (p=256, n=0, P=3916, N=4208)
IF odor = {a} THEN class = e (p=400, n=0, P=4208, N=3916)
IF odor = {l} THEN class = e (p=400, n=0, P=4208, N=3916)
IF odor = {c} THEN class = p (p=192, n=0, P=3916, N=4208)
IF odor = {m} THEN class = p (p=36, n=0, P=3916, N=42

In [40]:
CR_rules = []

for i in range(0, 8, 3):
    ruleset.rules[i].exception_rule = ruleset.rules[i + 2]
    ruleset.rules[i].reference_rule = ruleset.rules[i + 1]
    CR_rules.append(ruleset.rules[i])

for i in range(9, len(ruleset.rules)):
    CR_rules.append(ruleset.rules[i])

In [41]:
for rule in CR_rules:
    if rule.exception_rule is not None:
        GACE = calculate_GACE(rule, rule.exception_rule , return_ACE = False)
        RI = calculate_RI(rule, rule.exception_rule , rule.reference_rule)
        RES = calculate_my_measure(rule, rule.exception_rule)

        print(f"CR: {rule}, GACE: {round(GACE,3)}, RI: {round(RI,3)}, RES: {round(RES,3)}")
        print(f"RR: {rule.reference_rule} ")
        print(f"ER: {rule.exception_rule} ")
        print("")

    else:
        print(f"CR: {rule}")
        print("")

CR: IF odor = {n} THEN class = e (p=3408, n=120, P=4208, N=3916), GACE: 0.018, RI: 4.883, RES: 0.672
RR: IF stalk-color-above-ring = {y} THEN class = p (p=8, n=0, P=3916, N=4208) 
ER: IF odor = {n} AND stalk-color-above-ring = {y} THEN class = p (p=8, n=0, P=3916, N=4208) 

CR: IF stalk-root = {c} THEN class = e (p=512, n=44, P=4208, N=3916), GACE: 0.013, RI: 3.689, RES: 0.515
RR: IF ring-type = {n} THEN class = p (p=36, n=0, P=3916, N=4208) 
ER: IF stalk-root = {c} AND ring-type = {n} THEN class = p (p=36, n=0, P=3916, N=4208) 

CR: IF bruises = {t} THEN class = e (p=2752, n=624, P=4208, N=3916), GACE: 0.033, RI: 2.46, RES: 0.606
RR: IF spore-print-color = {r} THEN class = p (p=72, n=0, P=3916, N=4208) 
ER: IF bruises = {t} AND spore-print-color = {r} THEN class = p (p=72, n=0, P=3916, N=4208) 

CR: IF odor = {f} THEN class = p (p=2160, n=0, P=3916, N=4208)

CR: IF odor = {p} THEN class = p (p=256, n=0, P=3916, N=4208)

CR: IF odor = {a} THEN class = e (p=400, n=0, P=4208, N=3916)

CR

c:\Users\dawid\Desktop\exception_rules_repo\exception-rules\exception-rules\exception_rules\measures.py:11: RuntimeWarning: divide by zero encountered in log2
  not_X = cov.n/(cov.P + cov.N) * np.log2((cov.n/(cov.p+cov.n))/(cov.N/(cov.P+cov.N)))
c:\Users\dawid\Desktop\exception_rules_repo\exception-rules\exception-rules\exception_rules\measures.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  not_X = cov.n/(cov.P + cov.N) * np.log2((cov.n/(cov.p+cov.n))/(cov.N/(cov.P+cov.N)))


In [42]:
with open("txt_results/Ant_colony_rules.txt", "w", encoding="utf-8") as file:
    for rule in CR_rules:
        if rule.exception_rule is not None:
            GACE = calculate_GACE(rule, rule.exception_rule, return_ACE=False)
            RI = calculate_RI(rule, rule.exception_rule, rule.reference_rule)
            RES = calculate_my_measure(rule, rule.exception_rule)

            file.write(f"CR: {rule}, GACE: {round(GACE,3)}, RI: {round(RI,3)}, RES: {round(RES,3)}\n")
            file.write(f"RR: {rule.reference_rule} \n")
            file.write(f"ER: {rule.exception_rule} \n")
            file.write("\n")
        else:
            file.write(f"CR: {rule}\n")
            file.write("\n")